In [25]:
import xml.etree.cElementTree as ET
import pprint
import json
import re

# this program converts the xml file after standardizing the the document according to () and converts it into json.

### conversion rules :
# {
# "id": "2406124091",
# "type: "node", #this could also be "way"
# "visible":"true",
# "created": {
#           "version":"2",
#           "changeset":"17206049",
#           "timestamp":"2013-08-03T16:43:42Z",
#           "user":"linuxUser16",
#           "uid":"1219059"
#         },
# "pos": [41.9757030, -87.6921867],
# "address": {
#           "housenumber": "5157",
#           "postcode": "60625",
#           "street": "North Lincoln Ave"
#         },x yo/SPOT
# "amenity": "restaurant",
# "cuisine": "mexican",
# "name": "La Cabana De Don Luis",
# "phone": "1 (773)-271-5176"
# }


problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


osm_file = open("/Users/indur/Downloads/new-delhi_india.osm", "r")

json_fh = open("/Users/indur/Downloads/new-delhi_india.json","w")

counter = 0 


for event, element in ET.iterparse(osm_file, events=("start",)):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node["type"] = element.tag  
        created_hash = {}
        
        for k,v in element.attrib.iteritems():
            if k in [ "version",  "timestamp", "user", "uid"]:
                created_hash[k] = v
            elif k != "lat" and k != "lon":
                node[k] = v
            if k == "lat":
                node["position"] = [v, element.attrib["lon"]]
        
        node["created"] = created_hash
        
        address = {}
        amenity = []
        for e in element.iter("tag"):
            if problemchars.search(e.attrib["k"]):
                continue
                
            if e.attrib["k"].startswith("addr:"):
                e.attrib["k"] = e.attrib["k"].replace("addr:","")
                address[e.attrib["k"]] = e.attrib["v"]
                if address != {}:
                    node["address"] = address
            elif e.attrib["k"] == "amenity":
                amenity.append(e.attrib["v"])
            else:
                node[e.attrib["k"]] = e.attrib["v"]
                
        if amenity != []:
            node["amenity"] = amenity    
                

        if element.tag == "way":
            node_refs = []
            for e in element.iter("nd"):
                node_refs.append(e.attrib["ref"])
        if node_refs != []:
            node["node_refs"] = node_refs
            
        if node != {}:
            json_fh.write(json.dumps(node) + "\n")

json_fh.close()
